In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from hyperopt.pyll import scope
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np, xgboost as xgb
from xgboost import  XGBRegressor
from sklearn.model_selection import KFold, train_test_split



In [ ]:
# Set seed
SEED = 42
np.random.seed(SEED)

# Load data
data_path = 'C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv'
df = pd.read_csv(data_path)

#choose features and target
features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
target = 'Gas Used'

X = df[features].astype(float)
y = df[target].astype(float)

# divide train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

# noramalize
scaler_X = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler_X.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler_X.transform(X_test), columns=X.columns)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()


kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

# evaluate model
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled), start=1):
    X_tr = X_train_scaled.iloc[train_idx]
    y_tr = y_train_scaled[train_idx]
    X_val = X_train_scaled.iloc[val_idx]
    y_val = y_train_scaled[val_idx]

    model = XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10)

    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)

    mse  = mean_squared_error(y_val, y_pred)
    mae  = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_val, y_pred)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    print(f"Fold {fold} - MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

# outcome 
print("\n=== Kết quả trung bình (scaled) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")

Fold 1 - MSE: 0.000108 | MAE: 0.001475 | RMSE: 0.010409 | R²: 0.815865
Fold 2 - MSE: 0.000061 | MAE: 0.001485 | RMSE: 0.007819 | R²: 0.900478
Fold 3 - MSE: 0.000014 | MAE: 0.001351 | RMSE: 0.003780 | R²: 0.970408
Fold 4 - MSE: 0.000040 | MAE: 0.001471 | RMSE: 0.006305 | R²: 0.919190
Fold 5 - MSE: 0.000017 | MAE: 0.001394 | RMSE: 0.004091 | R²: 0.962422
Fold 6 - MSE: 0.000014 | MAE: 0.001385 | RMSE: 0.003799 | R²: 0.965351
Fold 7 - MSE: 0.000027 | MAE: 0.001448 | RMSE: 0.005153 | R²: 0.957949
Fold 8 - MSE: 0.000046 | MAE: 0.001467 | RMSE: 0.006748 | R²: 0.904991
Fold 9 - MSE: 0.000027 | MAE: 0.001388 | RMSE: 0.005165 | R²: 0.952265
Fold 10 - MSE: 0.000018 | MAE: 0.001419 | RMSE: 0.004232 | R²: 0.965730

=== Kết quả trung bình (scaled) ===
Avg MSE : 0.000037
Avg MAE : 0.001428
Avg RMSE: 0.005750
Avg R²  : 0.931465


In [ ]:
epochs = 20   
k = 10
cats = np.zeros((epochs, 4))  

for ep in range(epochs):
    kf = KFold(n_splits=k, shuffle=True, random_state=SEED + ep)

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_tr, X_va = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_tr, y_va = y_train_scaled[train_idx], y_train_scaled[val_idx]

        model = XGBRegressor()
            
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_va)

        mse  = mean_squared_error(y_va, y_pred)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(y_va, y_pred)
        r2   = r2_score(y_va, y_pred)

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    # saving metrics for each epoch
    cats[ep, :] = [
        np.mean(rmse_list),
        np.mean(mse_list),
        np.mean(mae_list),
        np.mean(r2_list)
    ]


final_mean = np.mean(cats, axis=0)

print(final_mean)  
print(cats)       

[4.75637426e-03 2.89492682e-05 7.81651915e-04 9.46648804e-01]
[[4.79775734e-03 2.92500879e-05 7.83133339e-04 9.46653191e-01]
 [5.83171560e-03 4.11827097e-05 7.95310246e-04 9.20422820e-01]
 [5.00556723e-03 2.93611350e-05 7.90027584e-04 9.46225089e-01]
 [4.42714400e-03 2.60314695e-05 7.78649110e-04 9.48566919e-01]
 [4.91506040e-03 2.87873859e-05 7.83250600e-04 9.44714024e-01]
 [4.36701578e-03 2.46280495e-05 7.81179060e-04 9.54100567e-01]
 [5.04861814e-03 3.00482851e-05 7.88728628e-04 9.44851354e-01]
 [5.44861130e-03 3.73954892e-05 7.95444970e-04 9.30280735e-01]
 [4.91372198e-03 2.95797548e-05 7.84476315e-04 9.45634900e-01]
 [4.67182030e-03 3.29321217e-05 7.77821120e-04 9.48911595e-01]
 [4.95017606e-03 3.22399830e-05 7.80595339e-04 9.39628940e-01]
 [4.27609788e-03 2.29980041e-05 7.68303356e-04 9.58130830e-01]
 [4.81666817e-03 2.81209352e-05 7.71775599e-04 9.46694478e-01]
 [4.53391459e-03 2.64238004e-05 7.79930085e-04 9.52321696e-01]
 [4.49715707e-03 2.67489697e-05 7.87132191e-04 9.5076634